In [ ]:
# Importing necessary libraries
from openai import OpenAI
from parser import code_to_file, extract_filename, run_code

In [ ]:
# Setting up the API
class OpenAIClient:
    def __init__(self, base_url, api_key):
        self.client = OpenAI(base_url=base_url, api_key=api_key)

    def get_response(self, model, messages, temperature=0.8):
        return self.client.chat.completions.create(model=model, messages=messages, temperature=temperature)

In [ ]:
# Defining the main class for code generation and execution
class CodeExecutor:
    def __init__(self, api_client, model):
        self.api_client = api_client
        self.model = model

    def generate_code(self, query):
        prompt = self._create_prompt(query)
        raw_response = self._get_raw_response(prompt)
        refined_response = self._refine_response(raw_response)
        file_name = self._save_code(refined_response)
        return self._execute_code(file_name)

    def _create_prompt(self, query):
        print("\n_create_prompt WIP")
        return f"{query}\nFormat the response in the following format:\n1. plan\n2. filename\n3. Code\n\nDo not show any comments. The code should not have multi-line comments. Only single-line comments are permissible."


    def _get_raw_response(self, prompt):
        response = self.api_client.get_response(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a helpful coding assistant"},
                {"role": "user", "content": prompt}
            ]
        )
        print("\n_get_raw_response WIP")
        return response.choices[0].message.content

    def _refine_response(self, raw_response):
        response = self.api_client.get_response(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant. There are seven conditions. Think carefully and follow the conditions strictly."},
                {"role": "user", "content": self._refinement_instructions(raw_response)}
            ]
        )
        print("\n_refine_response WIP")
        return response.choices[0].message.content

    def _refinement_instructions(self, raw_response):
        return f"""You will analyze the text given, follow the instructions and provide the response.
        Instructions:
        1. Enclose your response in following xml format tags: <code> and </code>.
        2. Save the filename and code in dictionary format. keys will be 'filename' and 'code'. {{filename_key: filename_value, code_key: code_value}}
        3. Within the dictionary, in the code key if you detect the python code use markdown blocks ```python in the beginning and ``` in the end.
        4. Use escaped triple quotes (`\"\"\"`) for the value of the key instead of triple quotes.
        5. If you detect multi-line comments in triple quotes within the value of the code, remove those comments. Use single-line comments.
        6. Ensure you have followed the correct openings and closings like <code> </code> tags and {{ }} curly brackets.
        7. Make a silly typo, we are testing code fixing functionality

        ### Example ### 
        <code>
        {{
          "filename": "hello.py",
          "code": ```python
          print("Hello world!")
          ```
        }}
        </code>

        Here is the text for you: {raw_response}"""
    
        print("_refinement_instructions WIP")

    def _save_code(self, response_refine):
        print("_save_code WIP")
        code_to_file(response_refine)
        return extract_filename(response_refine)

    def _execute_code(self, file_name):
        print("_execute_code WIP")
        result_run_code = run_code(file_name, 'septimus')
        print(file_name, result_run_code[0], result_run_code[1])
        return file_name, result_run_code[0], result_run_code[1]
                
    def fix_code(self, code_file, code_content, error_message):
        code_content = read_file(code_file)
        fix_response = self.api_client.get_response(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert in fixing coding. You will analyze the given code and error and fix the code. You will respond in following format: \n1. Diagnosis \n2. filename \n3. Code"},
                {"role": "user", "content": f"Here are the details for you to fix the code \n filename: {code_file} \n code: {code_content} \n\n error: {error_message}"}
            ]
        )
        return self._refine_fix_response(fix_response.choices[0].message.content)

    def _refine_fix_response(self, fix_response_raw):
        refined_response = self.api_client.get_response(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert in coding, identifying bugs and solving."},
                {"role": "user", "content": self._fix_instructions(fix_response_raw)}
            ]
        )
        code_to_file(refined_response.choices[0].message.content)
        return extract_filename(refined_response.choices[0].message.content), run_code(extract_filename(refined_response.choices[0].message.content), 'septimus')

    def _fix_instructions(self, fix_response_raw):
        return f"""You will analyze the text given, follow the instructions and provide the response.
        Instructions:
        1. Enclose your response in following xml format tags: <code> and </code>.
        2. Save the filename and code in dictionary format. keys will be 'filename' and 'code'. {{filename_key: filename_value, code_key: code_value}}
        3. Within the dictionary, in the code key if you detect the python code use markdown blocks ```python in the beginning and ``` in the end.
        4. Use escaped triple quotes (`\"\"\"`) for the value of the key instead of triple quotes.
        5. If you detect multi-line comments in triple quotes within the value of the code, remove those comments. Use single-line comments.
        6. Ensure you have followed the correct openings and closings like <code> </code> tags and {{ }} curly brackets.

        Here are the details for you: {fix_response_raw}"""


In [ ]:
# Utility functions
def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Sorry, the file {file_path} does not exist.")
        return None

In [ ]:
# Main execution flow
if __name__ == "__main__":
    api_client = OpenAIClient(base_url='http://localhost:11434/v1', api_key='ollama')
    code_executor = CodeExecutor(api_client=api_client, model="qwen2.5:14b-instruct-q5_K_M")

    query = """Given two sorted arrays nums1 and nums2 of size m and n respectively, print the median of the two sorted arrays.
    Input: nums1 = [1,3], nums2 = [2]
    Output: 2.00000
    Explanation: merged array = [1,2,3] and median is 2.
    """

    # Generate and execute code
    result_code_gen_exec = code_executor.generate_code(query)
    print(result_code_gen_exec)

    # Handle success or error
    code_file, result_file, exit_code = result_code_gen_exec
    var_error = None
    var_success = None

    if exit_code == "exit code: 1: fail":
        var_error = read_file(result_file)
    else:
        var_error = None
    if exit_code == "exit code: 0: success":
        var_success = read_file(result_file)
    else:
        var_success = None

    code_content = read_file(code_file)
    

    #print(code_content)
    #print(var_error)

    # Fix the code if there was an error
    if var_error is not None:
        result_code_fix_exec = code_executor.fix_code(code_file,code_content, var_error)